In [7]:
# Setup model connection
from dotenv import load_dotenv
from openai import OpenAI

from converter.converter import *

load_dotenv("../.env")
client = OpenAI(
    api_key=os.getenv("VLLM_API_KEY"),
    base_url="http://134.76.18.30:8082/v1"
)
model = "meta-llama/Llama-3.3-70B-Instruct"

## Load original dataset

In [8]:
from pprint import pprint

import pandas as pd
import spacy
from datasets import load_dataset, Dataset, DatasetDict
from tqdm import tqdm

# Load spaCy for POS tagging
nlp = spacy.load("en_core_web_sm")

tqdm.pandas()

selected_split = "test"
dataset = load_dataset("cais/mmlu", "all")[selected_split]
modified_dataset_name = "MMLU-NoOp-Plus"

hf_username = "LFrancis"
repo_id = f"{hf_username}/{modified_dataset_name}"

df = pd.DataFrame(dataset)
dataset_dict = DatasetDict()
dataset_dict["all"] = Dataset.from_pandas(df)
dataset_dict

DatasetDict({
    all: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 14042
    })
})

## Paraphrase Type: Naive Addition

In [9]:
dataset_dict["all_naive"] = convert_naive(pd.DataFrame(dataset_dict["all"]))
pprint(list(dataset_dict.keys()))

Rows mentioning Sebastian: 0
Rows mentioning 'icecream': 0


100%|██████████| 14042/14042 [00:00<00:00, 31725.24it/s]

['all', 'all_naive']


In [10]:
upload(dataset_dict, repo_id)

Uploading all


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Uploading all_naive


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

No files have been modified since last commit. Skipping to prevent empty commit.


## Paraphrase Type: Addition

In [11]:
dataset_dict["all_addition"] = convert_additional(pd.DataFrame(dataset_dict["all"]), client, model, nlp, )
pprint(list(dataset_dict.keys()))

  8%|▊         | 1139/14042 [15:10<2:51:59,  1.25it/s]


KeyboardInterrupt: 

In [ ]:
upload(dataset_dict, repo_id)

## Paraphrase Type: Lexicon-Changes


In [ ]:
dataset_dict["all_lexicon"] = convert_lexicon(pd.DataFrame(dataset_dict["all"]), client, model, nlp)
dataset_dict.keys()

In [ ]:
upload(dataset_dict, repo_id)

## Paraphrase Type: Syntax-Changes


In [ ]:
dataset_dict["all" + "_syntax"] = convert_syntax(pd.DataFrame(dataset_dict["all"]), nlp)
dataset_dict.keys()

In [ ]:
upload(dataset_dict, repo_id)

## Paraphrase Type: Typo

In [ ]:
dataset_dict["all" + "_typo"] = convert_typo(pd.DataFrame(dataset_dict["all"]), nlp)
dataset_dict.keys()

In [ ]:
upload(dataset_dict, repo_id)

## Paraphrase Type: Scramble

In [ ]:
dataset_dict["all" + "_scramble"] = convert_scramble(pd.DataFrame(dataset_dict["all"]), nlp)
dataset_dict.keys()

In [ ]:
upload(dataset_dict, repo_id)